In [1]:
!pip install tensorflow pandas numpy matplotlib astropy pydl statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 4.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 4.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.9 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 4.5 MB/s eta 0:00:000:00:01m eta 0:00:01


In [ ]:
import os
import tensorflow as tf

# tces_file = '/mnt/tess/astronet/tces-vetting-v4-toi-train.csv'
# file_pattern = '/mnt/tess/astronet/tfrecords-vetting-5-toi-train/*'
# model_name = 'AstroCNNModelVetting'
# config_name = 'vrevised'
# labels = ['p', 'e', 'n']
tces_file = '/home/astronet/NASA/tces_with_labels_v3.csv'
file_pattern = '/home/astronet/astronet/tfrecords-new+old/*'
model_name = 'AstroCNNModel'
config_name = 'final_alpha_1'
labels = ['E', 'N', 'J', 'S', 'B']

filenames = tf.io.gfile.glob(file_pattern)
    
filenames

2025-10-01 13:21:45.452422: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-01 13:21:45.900881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-01 13:21:46.141032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-01 13:21:46.216674: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-01 13:21:46.650456: I tensorflow/core/platform/cpu_feature_guar

['/home/astronet/astronet/tfrecords-new+old/00000-of-00008']

In [3]:
series = {}

for filename in filenames:
  tfr = tf.data.TFRecordDataset(filename)
  num_records = 0
  for record in tfr:
    num_records += 1
    ex = tf.train.Example.FromString(record.numpy())
    for k in ex.features.feature.keys():
      f = ex.features.feature[k]
      if f.int64_list.value:
        v = f.int64_list.value[0]
      elif f.float_list.value:
        v = f.float_list.value[0]
      elif f.bytes_list.value:
        v = f.bytes_list.value[0].decode()
      else:
        continue

      if k not in series:
        series[k] = []
      series[k].append(v)
  print(filename, num_records)

/home/astronet/astronet/tfrecords-new+old/00000-of-00008 0


I0000 00:00:1759324912.937969   11874 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1759324913.487376   11874 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1759324913.487423   11874 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1759324913.491714   11874 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1759324913.491771   11874 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [13]:
from matplotlib import pyplot as plt

counts = [sum(examples_table['disp_{}'.format(l)] > 0) for l in labels]
ax = plt.bar(labels, counts)
for i in range(len(labels)):
    b = ax[i]
    height = b.get_height()
    x, y = b.get_xy()
    plt.annotate(
        '{} - {:.0%}'.format(counts[i], counts[i] / sum(counts)),
        (x + 0.1, y + height + 11))

KeyError: 'disp_E'

In [5]:
import pandas as pd

examples_table = pd.DataFrame.from_dict(series)

pd.set_option('display.max_columns', None)
# examples_table[['secondary_scale']].describe()
examples_table.describe()

ValueError: Cannot describe a DataFrame without columns

In [14]:
examples_table.head(3)

""


In [17]:
tce_table[tce_table.index == 8209]

,TIC ID,Epoch,Period,Duration,Depth,SMass,SRad,SRadEst,Tmag,Year,MinT,MaxT,Split,Consensus Label,L1,L2,L3,L4,L5,L6,L7,L8,Notes
8209,272189849.0,1339.390733,27.323687,0.3502896673,570,NaN,11.769,10.59198264,10.7538,1.0,1325.321768,1652.86583,train,J,J,J,J,J,J,NaN,NaN,NaN,NaN


In [18]:
import numpy as np

print('Label mismtaches between TCE and tfrecords:')
np.array(set(tce_table[tce_table[f'disp_{labels[0]}'] > 0]['Astro ID'].values)
    - set(examples_table[examples_table[f'disp_{labels[0]}'] > 0]['astro_id'].values))

Label mismtaches between TCE and tfrecords:


KeyError: 'disp_E'

In [30]:
import tensorflow as tf

from astronet import models
from astronet.astro_cnn_model import input_ds

config = models.get_model_config(model_name, config_name)

ds = input_ds.build_dataset(
      file_pattern=file_pattern,
      input_config=config.inputs,
      batch_size=1,
      include_labels=False,
      shuffle_filenames=False,
      repeat=1,
      include_identifiers=True)
labels_ds = input_ds.build_dataset(
      file_pattern=file_pattern,
      input_config=config.inputs,
      batch_size=1,
      include_labels=True,
      shuffle_filenames=False,
      repeat=1,
      include_identifiers=True)
labels_iter = iter(labels_ds)

label_index = {k.lower(): i for i, k in enumerate(config.inputs.label_columns)}
cols = ["disp_E", "disp_N", "disp_J", "disp_S", "disp_B"]

rec = tce_table[tce_table['TIC ID'] == 36279]
print(rec["disp_E"].values[0])

all_ids = []
bad_labels = []
for d in ds:
  lab = next(labels_iter)
  
  def lam(e):
    if e.dtype == tf.int64:
        return e
    if tf.reduce_any(tf.math.is_nan(e)):
        tf.print(e, summarize=-1)
        raise ValueError('data has NaNs.')
    return e
  ex_id = d[1].numpy().item()
  all_ids.append(ex_id)
  
  assert lab[0]['duration'] == d[0]['duration']
  rec = tce_table[tce_table['Astro ID'] == ex_id]
  
  for c in cols:
    if (lab[1][0][label_index[c.lower()]].numpy() == 0) != (rec[c].values[0] == 0):
      bad_labels.append(ex_id)
      print('bad example: ', ex_id)
      print(rec)
      print(cols)
      print(lab[1][0])
      break
  if bad_labels:
    break
  
  try:
    tf.nest.map_structure(lam, d)
  except ValueError as e:
    print(e)
    print(d[1])
    break
else:
  print('No NaNs or mismtached labels found.')

if len(all_ids) == len(set(all_ids)):
  print('No duplicates found.')
else:
  print('Found duplicates!', len(all_ids) - len(set(all_ids)))
  print([t for t in set(all_ids) if all_ids.count(t) > 1])

KeyError: 'disp_E'

In [7]:
def astro_id(tic_id):
  return tce_table[tce_table['TIC ID'] == tic_id]['Astro ID'].values[0]

In [8]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from astronet.preprocess import preprocess


tess_data_dir = '/home/astronet/NASA/LC/LC'

def find_tce(astro_id):
  with tf.device('cpu'):
    for filename in filenames:
      tfr = tf.data.TFRecordDataset(filename)
      for record in tfr:
        ex = tf.train.Example.FromString(record.numpy())
        if (ex.features.feature["astro_id"].int64_list.value[0] == astro_id):
          print('TIC ID:', tic_id)
          for l in labels:
              print(f'{l}:', ex.features.feature[f"disp_{l}"].int64_list.value[0])
          print('Duration:', ex.features.feature["Duration"].float_list.value[0])
          return ex

    raise ValueError("{} not found in files: {}".format(astro_id, filenames))

In [9]:
ds = ds.cache()
def plot_ds_tce(ds, astro_id):
    for d in ds:
        if d[1] == astro_id:
            for k, v in d[0].items():
                if k.startswith('local_'):
                    continue
                if k.startswith('global_'):
                    continue
                if k.startswith('secondary_'):
                    continue
                if k.startswith('sample_'):
                    continue
                print(f'{k:25}: {v.numpy()}')
            global_view = np.array(d[0]['global_view'][0].numpy())
            local_view = np.array(d[0]['local_view'][0].numpy())
            secondary_view = np.array(d[0]['secondary_view'][0].numpy())
            fig, axes = plt.subplots(2, 3, figsize=(20, 12))
            axes[0, 0].plot(global_view, '.-')
            axes[0, 1].plot(local_view, '.-')
            axes[0, 2].plot(secondary_view, '.-')
            axes[1, 0].plot(d[0]['global_mask'][0].numpy(), '.-')
            axes[1, 1].plot(d[0]['global_view_0.3'][0].numpy(), '.-')
            axes[1, 2].plot(d[0]['global_view_5.0'][0].numpy(), '.-')
            plt.show()
            plt.close('all')
            return

In [10]:
tic_id = 349412074
plot_ds_tce(ds, astro_id(tic_id))

KeyError: 'Astro ID'

In [30]:
tce_table[tce_table['TIC ID'] == tic_id]

,TIC ID,Epoch,Period,Duration,Depth,SMass,SRad,SRadEst,Tmag,Year,MinT,MaxT,Split,Consensus Label,L1,L2,L3,L4,L5,L6,L7,L8,Notes
22607,349412074.0,1326.354909,10.366559,0.1458574851,250,NaN,8.03096,8.315433067,9.6103,1.0,1325.320719,1652.866044,val,E,E,J,E,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
astro_id(tic_id)

KeyError: 'Astro ID'

In [33]:
examples_table[examples_table['astro_id'] == astro_id(tic_id)]

KeyError: 'astro_id'

In [34]:
!ls /mnt/tess/lc-v | grep 237320326

ls: cannot access '/mnt/tess/lc-v': No such file or directory


In [35]:
tic_id = 334227600
tce = find_tce(astro_id(tic_id))

list(tce.features.feature.keys())

KeyError: 'Astro ID'

In [36]:
print(filenames)


['/home/astronet/astronet/tfrecords-new+old/00000-of-00008']


In [46]:
import pandas as pd

tce_table = pd.read_csv(
    tces_file,
    comment="#",   # ignore metadata lines
    skiprows=[1],  # skip the units row
)
print(len(tce_table))
tce_table.head(3)

24927


,TIC ID,Epoch,Period,Duration,Depth,SMass,SRad,SRadEst,Tmag,Year,MinT,MaxT,Split,Consensus Label,L1,L2,L3,L4,L5,L6,L7,L8,Notes
0,NaN,BTJD,days,days,ppm,M_sun,R_sun,R_sun,NaN,NaN,BTJD,BTJD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,387242167.0,1655.407925,4.081254,0.1808811773,330,NaN,11.2034,11.46885329,6.7236,1.0,1653.925888,1682.342692,test,NaN,J,N,J,NaN,NaN,NaN,NaN,NaN,NaN
2,141709436.0,1338.471105,24.959872,0.2188980774,640,0.98,0.886142,0.8787521896,6.9129,1.0,1325.322344,1682.344195,test,J,J,J,J,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

# Ensure TIC ID is int (drop NaNs first)
tce_table = tce_table.dropna(subset=['TIC ID'])
tce_table['TIC ID'] = tce_table['TIC ID'].astype(int)

# Define one-hot columns
cols = ["disp_E", "disp_N", "disp_J", "disp_S", "disp_B"]

# Create empty one-hot DataFrame with dtype=int
one_hot = pd.DataFrame(0, index=tce_table.index, columns=cols, dtype=int)

# Fill in 1s according to Consensus Label
for i, label in enumerate(tce_table['Consensus Label']):
    if pd.notna(label):
        col_name = f"disp_{label}"
        if col_name in one_hot.columns:
            one_hot.at[i, col_name] = 1

# Attach one-hot columns to original table
tce_table = pd.concat([tce_table, one_hot], axis=1)

TIC ID               int64
Epoch               object
Period              object
Duration            object
Depth               object
SMass               object
SRad                object
SRadEst             object
Tmag               float64
Year               float64
MinT                object
MaxT                object
Split               object
Consensus Label     object
L1                  object
L2                  object
L3                  object
L4                  object
L5                  object
L6                  object
L7                  object
L8                  object
Notes               object
disp_E               int64
disp_N               int64
disp_J               int64
disp_S               int64
disp_B               int64
dtype: object
      TIC ID        Epoch     Period      Duration  Depth SMass      SRad  \
1  387242167  1655.407925   4.081254  0.1808811773    330   NaN   11.2034   
2  141709436  1338.471105  24.959872  0.2188980774    640  0.98  0.886142   

In [48]:
# Remove rows where TIC ID is NaN
tce_table = tce_table.dropna(subset=['TIC ID'])

# Then apply your function
def make_file_name(tic_id):
    tic_id_int = int(tic_id)
    tic_id_str = str(tic_id_int).zfill(16)
    return f"astronet_tess_ffi_s0039-{tic_id_str}_tess_v01_llc.fits"

tce_table['File'] = tce_table['TIC ID'].apply(make_file_name)

# Check result
print(tce_table.columns)


Index(['TIC ID', 'Epoch', 'Period', 'Duration', 'Depth', 'SMass', 'SRad',
       'SRadEst', 'Tmag', 'Year', 'MinT', 'MaxT', 'Split', 'Consensus Label',
       'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'Notes', 'disp_E',
       'disp_N', 'disp_J', 'disp_S', 'disp_B', 'File'],
      dtype='object')


In [49]:
output_path = "/home/astronet/tces-new+old.csv"
tce_table.to_csv(output_path, index=False)
print(f"Saved processed table to {output_path}")

Saved processed table to /home/astronet/tces-new+old.csv
